# NEURAL NETWORK with K-MEANS for MNIST

#### Import

In [1]:
import sys
import time
import pickle
import gzip
from random import randint
from scipy import misc
from scipy import special
import scipy.ndimage
from scipy.sparse import csr_matrix
import numpy as np
import datetime as dt
from sklearn.cluster import KMeans,MiniBatchKMeans
import matplotlib.pyplot as plt
import json
import collections
import math
import sys

#### Initial Settings

In [2]:
DATA_PATH = 'data/mnist/'

IMAGES_TRAIN = 'data_training'
IMAGES_TEST = 'data_testing'

RANDOM_SEED = 42
N_CLASSES = 10
N_FEATURES = 28 * 28

In [3]:
data_training = DATA_PATH+IMAGES_TRAIN
data_testing = DATA_PATH+IMAGES_TEST
ft = gzip.open(data_training, 'rb')
TRAINING = pickle.load(ft)
ft.close()
ft = gzip.open(data_testing, 'rb')
TESTING = pickle.load(ft)
ft.close()

## Neural Network

In [4]:
class Neural_Network:

    # Stop function
    # 0 : Fixed (stop after n epochs)
    # 1 : Progress (stop after n epochs w/o improvements)
    # 2 : Std Dev (stop after an improvements below n)
    
    def __init__(self, neurons, batchsize, stop_function, stop_parameter):
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.batchsize = batchsize
        self.stop_f = stop_function
        self.stop_p = stop_parameter
        self.best = 0.
        self.same = 0
        self.iteration = 0
        
        # Standardize random weights
        np.random.seed(RANDOM_SEED)
        hidden_layer = np.random.rand(self.neurons, self.input_size + 1) / self.neurons
        output_layer = np.random.rand(self.output_size, self.neurons + 1) / self.output_size
        self.layers = [hidden_layer, output_layer]

    def train(self, training, testing):
        
        accu = [0.,0.]
        
        # Batch Setting
        len_batch_train = len(training[0])
        len_batch_test = len(testing[0])
        if(self.batchsize > 0 and self.batchsize <= 1):
            len_batch_train = int(np.ceil(len_batch_train * self.batchsize))
            len_batch_test = int(np.ceil(len_batch_test * self.batchsize))
        
        # Start prints 
        self.start_time = dt.datetime.now()
        print('-- Training Session Start (%s) --' % (self.start_time))
        typeTrainingPrint = "Stop Function: "    
        if self.stop_f == 0:
            typeTrainingPrint += str(self.stop_p)+" epochs"
        elif self.stop_f == 1:
            typeTrainingPrint += str(self.stop_p)+" epoch(s) w/o improvements"
        elif self.stop_f == 2:
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        print('\nNeurons: %d\nBatch Train: %d\nBatch Test: %d\n%s\n' % (self.neurons,len_batch_train,len_batch_test,typeTrainingPrint))
        
        # Divide training and testing batches
        test_output = testing[0:len_batch_test][0:len_batch_test]
        inputs = training[0][0:len_batch_train]
        targets = np.zeros((len_batch_train, 10))
        for i in range(len_batch_train):
            targets[i, training[1][i]] = 1

        # Performs iterations
        while not self.is_stop_function_enabled(accu[1]):
            for input_vector, target_vector in zip(inputs, targets):
                self.backpropagate(input_vector, target_vector)
            
            # Accuracy
            accu = self.accu(test_output)
            self.iteration += 1
            
            # Messages
            if (self.iteration == 1 or self.iteration % 10 == 0):
                self.print_message_iter(self.iteration,accu,self.ETAepoch(self.start_time))
                
        # Print last epoch
        if (self.iteration % 10 != 0):
            self.print_message_iter(self.iteration,accu,self.ETAepoch(self.start_time))

        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))

    def feed_forward(self, input_vector):
        outputs = []
        for layer in self.layers:
            input_with_bias = np.append(input_vector, 1)   # Ajout constante
            output = np.inner(layer, input_with_bias)
            output = special.expit(output)
            outputs.append(output)
            # The output is the input of the next layer
            input_vector = output
        return outputs

    def backpropagate(self, input_vector, target):
        c = 1./math.sqrt(self.iteration + 10)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector)

        # Calculation of partial derivatives for the output layer and subtraction
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        self.layers[-1] -= c*np.outer(output_deltas, np.append(hidden_outputs, 1))

        # Calculation of partial derivatives for the hidden layer and subtraction
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * np.dot(np.delete(self.layers[-1], 200, 1).T, output_deltas)
        self.layers[0] -= c*np.outer(hidden_deltas, np.append(input_vector, 1))

    def predict(self, input_vector):
        return self.feed_forward(input_vector)[-1]

    def predict_one(self, input_vector):
        return np.argmax(self.feed_forward(input_vector)[-1])

    def accu(self, testing):
        res = np.zeros((10, 2))
        for k in range(len(testing[1])):
            if self.predict_one(testing[0][k]) == testing[1][k]:
                res[testing[1][k]] += 1
            else:
                res[testing[1][k]][1] += 1
        total = np.sum(res, axis=0)
        each = [res[k][0]/res[k][1] for k in range(len(res))]
        min_c = sorted(range(len(each)), key=lambda k: each[k])[0]
        return np.round([each[min_c]*100, total[0]/total[1]*100, min_c], 2)
    
    def is_stop_function_enabled(self,accuracy):
        if self.stop_f == 0:
            if self.iteration < self.stop_p:
                return False
            else:
                return True
        elif self.stop_f == 1:
            if accuracy > self.best or self.iteration == 0:
                self.same = 0
                self.best = accuracy
                return False
            else:
                self.same += 1
                if self.same < self.stop_p:
                    return False
                else:
                    return True
        elif self.stop_f == 2:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
    
    def print_message_iter(self,iteration,accu,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "
        message += 'Accuracy: '+str(accu[1]).zfill(4)+'%\tMin: '+ str(accu[0]).zfill(4)+ '% ('+str(int(accu[2]))+')'
        print(message)
    
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
        
    def getWeights(self):
        return self.layers

In [5]:
nn = Neural_Network(neurons=300,batchsize=1,stop_function=1,stop_parameter=3)
nn.train(TRAINING,TESTING)

-- Training Session Start (2018-11-17 17:05:13.133558) --

Neurons: 300
Batch Train: 60000
Batch Test: 10000
Stop Function: 3 epoch(s) w/o improvements

Epoch 001 (39s)    Accuracy: 82.49%	Min: 12.44% (5)
Epoch 010 (6m30s)  Accuracy: 96.27%	Min: 94.28% (5)
Epoch 020 (13m2s)  Accuracy: 97.16%	Min: 95.28% (8)
Epoch 026 (16m59s) Accuracy: 97.31%	Min: 96.13% (9)

-- Training Session End (2018-11-17 17:22:12.733445) --


### Cluster K-Means

In [6]:
def nearest_centroid_index(centers,value):
    centers = np.asarray(centers)
    idx = (np.abs(centers - value)).argmin()
    return idx

In [7]:
def build_clusters(cluster,weights):
    kmeans = MiniBatchKMeans(n_clusters=cluster,random_state=RANDOM_SEED)
    kmeans.fit(np.hstack(weights).reshape(-1,1))
    return kmeans.cluster_centers_

### Matrix (Helper Function)

In [8]:
def redefine_weights(weights,centers):
    arr_ret = np.empty_like(weights).astype(np.int16)
    for i, row in enumerate(weights):
        for j, col in enumerate(row):
            arr_ret[i,j] = nearest_centroid_index(centers,weights[i,j])
    return arr_ret

In [9]:
def idx_matrix_to_matrix(idx_matrix,centers,shape):
    return centers[idx_matrix.reshape(-1,1)].reshape(shape)

In [10]:
def centroid_gradient_matrix(idx_matrix,gradient,cluster):
    return scipy.ndimage.sum(gradient,idx_matrix,index=range(cluster))

## Neural Network with K-Means

In [11]:
class Neural_Network_KM:

    def __init__(self, neurons, batchsize, cluster, pre_weights, verbose, stop_function, stop_parameter):
        
        start_setting_time = dt.datetime.now()
        
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.batchsize = batchsize
        self.verbose = verbose
        self.cluster = cluster
        self.iteration = 0
        self.stop_f = stop_function
        self.stop_p = stop_parameter
        self.best = 0.
        self.same = 0
        
        # Variable for shape
        shape_hidden = (self.neurons,self.input_size+1)
        shape_output = (self.output_size,self.neurons+1)
        self.layers_shape = [shape_hidden,shape_output]
            
        # Initialize cluster for pre-trained weights (dict with centers)
        c_hidden = build_clusters(self.cluster,pre_weights[0])
        c_output = build_clusters(self.cluster,pre_weights[-1])
        self.centers = [c_hidden,c_output]
        
        # Initialize index matrix for pre-trained weights
        idx_hidden = redefine_weights(pre_weights[0],self.centers[0])
        idx_output = redefine_weights(pre_weights[-1],self.centers[-1])
        self.idx_layers = [idx_hidden,idx_output]
        
        # Setting time print    
        end_setting_time = dt.datetime.now() - start_setting_time
        eta = divmod(end_setting_time.days * 86400 + end_setting_time.seconds, 60)
        self.eta_print_setting = str(eta[0])+"m"+str(eta[1])+"s"
        if self.verbose:
            print("--- Setting Time: %s ---" % self.eta_print_setting)
    
 

    def train(self, training, testing):
        
        accu = [0.,0.]
        
        # Batch Setting
        len_batch_train = len(training[0])
        len_batch_test = len(testing[0])
        if(self.batchsize > 0 and self.batchsize <= 1):
            len_batch_train = int(np.ceil(len_batch_train * self.batchsize))
            len_batch_test = int(np.ceil(len_batch_test * self.batchsize))
        
        # Divide training and testing batches
        test_output = testing[0:len_batch_test][0:len_batch_test]
        inputs = training[0][0:len_batch_train]
        targets = np.zeros((len_batch_train, 10))
        for i in range(len_batch_train):
            targets[i, training[1][i]] = 1
        
        # Start prints 
        self.start_time = dt.datetime.now()
        print('-- Training Session Start (%s) --' % (self.start_time))
        typeTrainingPrint = "Stop Function: "    
        if self.stop_f == 0:
            typeTrainingPrint += str(self.stop_p)+" epochs"
        elif self.stop_f == 1:
            typeTrainingPrint += str(self.stop_p)+" epoch(s) w/o improvements"
        elif self.stop_f == 2:
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        print('\nNeurons: %d\nClusters: %d\nBatch Train: %d\nBatch Test: %d\n%s\n' % (self.neurons,self.cluster,len_batch_train,len_batch_test,typeTrainingPrint))
        
        # Performs iterations
        while not self.is_stop_function_enabled(accu[1]):
            
            # Backpropagate with feed forward
            for input_vector, target_vector in zip(inputs, targets):
                weights = []
                for i,c,s in zip(self.idx_layers,self.centers,self.layers_shape):
                    w = idx_matrix_to_matrix(i,c,s)
                    weights.append(w)
                self.backpropagate(input_vector, target_vector, weights)
                
            # Accuracy
            accu = self.accu(test_output,weights)
            self.iteration += 1
            
            # Messages
            self.print_message_iter(self.iteration,accu,self.ETAepoch(self.start_time))
                      
        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))

    def feed_forward(self, input_vector, weights):
        outputs = []
        for w in weights:
            input_with_bias = np.append(input_vector, 1)   # Ajout constante
            output = np.inner(w, input_with_bias)
            output = special.expit(output) # Sigmoid function
            outputs.append(output)
            # The output is the input of the next layer
            input_vector = output
        return outputs

    def backpropagate(self, input_vector, target, weights):
        c = 1./math.sqrt(self.iteration + 10)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector, weights)

        # Calculation of partial derivatives for the output layer and subtraction
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        gradient = np.outer(output_deltas, np.append(hidden_outputs, 1))
        cg = centroid_gradient_matrix(self.idx_layers[-1],gradient,self.cluster)
        self.centers[-1] = self.centers[-1] - c * np.array(cg).reshape(self.cluster,1)

        # Calculation of partial derivatives for the hidden layer and subtraction
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * np.dot(np.delete(weights[-1], 300, 1).T, output_deltas)
        gradient = np.outer(hidden_deltas, np.append(input_vector, 1))
        cg = centroid_gradient_matrix(self.idx_layers[0],gradient,self.cluster)
        self.centers[0] = self.centers[0] - c * np.array(cg).reshape(self.cluster,1)
        
    
    
    def predict(self, input_vector, weights):
        return self.feed_forward(input_vector,weights)[-1]

    def predict_one(self, input_vector, weights):
        return np.argmax(self.feed_forward(input_vector,weights)[-1])

    def accu(self, testing, weights):
        res = np.zeros((10, 2))
        for k in range(len(testing[1])):
            if self.predict_one(testing[0][k], weights) == testing[1][k]:
                res[testing[1][k]] += 1
            else:
                res[testing[1][k]][1] += 1
        total = np.sum(res, axis=0)
        each = [res[k][0]/res[k][1] for k in range(len(res))]
        min_c = sorted(range(len(each)), key=lambda k: each[k])[0]
        return np.round([each[min_c]*100, total[0]/total[1]*100, min_c], 2)
    
    
    def is_stop_function_enabled(self,accuracy):
        if self.stop_f == 0:
            if self.iteration < self.stop_p:
                return False
            else:
                return True
        elif self.stop_f == 1:
            if accuracy > self.best or self.iteration == 0:
                self.same = 0
                self.best = accuracy
                return False
            else:
                self.same += 1
                if self.same < self.stop_p:
                    return False
                else:
                    return True
        elif self.stop_f == 2:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
    
    def print_message_iter(self,iteration,accu,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "
        message += 'Accuracy: '+str(accu[1]).zfill(4)+'%\tMin: '+ str(accu[0]).zfill(4)+ '% ('+str(int(accu[2]))+')'
        print(message)
        
    def getWeights(self):
        return self.layers
    
    def minsec2sec(self,time):
        if 'm' in time:
            splitted = time.split('m')
            return int(splitted[0]) * 60 + int(splitted[1][:-1])
        else:
            return int(time[:-1])
    
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret

In [12]:
pre_trained_weights = nn.getWeights()
nn_km = Neural_Network_KM(neurons=300,batchsize=1,cluster=256,pre_weights=pre_trained_weights,verbose=True,stop_function=1,stop_parameter=2)
nn_km.train(TRAINING,TESTING)

--- Setting Time: 0m2s ---
-- Training Session Start (2018-11-17 17:22:15.887921) --

Neurons: 300
Clusters: 256
Batch Train: 60000
Batch Test: 10000
Stop Function: 2 epoch(s) w/o improvements

Epoch 001 (2m47s)  Accuracy: 96.68%	Min: 94.09% (2)
Epoch 002 (5m31s)  Accuracy: 96.74%	Min: 93.12% (8)
Epoch 003 (8m16s)  Accuracy: 96.87%	Min: 94.35% (9)
Epoch 004 (11m6s)  Accuracy: 96.58%	Min: 94.85% (9)
Epoch 005 (14m0s)  Accuracy: 97.01%	Min: 95.04% (9)
Epoch 006 (16m52s) Accuracy: 96.74%	Min: 94.67% (2)
Epoch 007 (19m45s) Accuracy: 96.81%	Min: 94.17% (5)

-- Training Session End (2018-11-17 17:42:00.901200) --


### Pruning

In [13]:
def pruning_matrix(mat,perc):
    real_perc = math.floor(50 + perc / 2)
    percentile = np.percentile(mat,real_perc)
    w_pruned = np.copy(mat)
    for i,row in enumerate(mat):
        for j,_ in enumerate(row):
            if abs(mat[i,j]) > percentile:
                w_pruned[i,j] = 0
    return csr_matrix(w_pruned)

In [14]:
def P_build_clusters(cluster,weights):
    kmeans = MiniBatchKMeans(n_clusters=cluster,random_state=RANDOM_SEED)
    kmeans.fit(weights.data.reshape(-1,1))
    return kmeans.cluster_centers_

In [15]:
def P_redefine_weights(weights,centers):
    csr_idx = weights.copy()
    arr_ret = np.empty_like(csr_idx.data).astype(np.int16)
    for i,w in enumerate(csr_idx.data):
        arr_ret[i] = nearest_centroid_index(centers,w)
    csr_idx.data = arr_ret
    return csr_idx

In [16]:
def P_idx_matrix_to_matrix(idx_matrix,centers):
    w_csr = idx_matrix.copy()
    w_csr.data = centers[w_csr.data]
    return w_csr

In [17]:
def P_centroid_gradient_matrix(idx_matrix,gradient,cluster):
    return scipy.ndimage.sum(gradient,idx_matrix.todense(),index=range(cluster))

## Neural Network with Pruning and K-Means

In [18]:
class Neural_Network_PR_KM:

    def __init__(self, neurons, batchsize, cluster, pre_weights, percentile, verbose, stop_function, stop_parameter):
        
        start_setting_time = dt.datetime.now()
        
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.batchsize = batchsize
        self.percentile = percentile
        self.verbose = verbose
        self.cluster = cluster
        self.iteration = 0
        self.stop_f = stop_function
        self.stop_p = stop_parameter
        self.best = 0.
        self.same = 0
        
        # Pruning weights
        pw_hidden = pruning_matrix(pre_weights[0],self.percentile)
        pw_output = pruning_matrix(pre_weights[1],self.percentile)
        self.pruned_weights = [pw_hidden,pw_output]
        
        # Variable for shape
        shape_hidden = (self.neurons,self.input_size+1)
        shape_output = (self.output_size,self.neurons+1)
        self.layers_shape = [shape_hidden,shape_output]
            
        # Initialize cluster for pre-trained weights pruned
        c_hidden = P_build_clusters(self.cluster,self.pruned_weights[0])
        c_output = P_build_clusters(self.cluster,self.pruned_weights[-1])
        self.centers = [c_hidden,c_output]
        
        # Initialize index matrix for pre-trained weights
        idx_hidden = P_redefine_weights(self.pruned_weights[0],self.centers[0])
        idx_output = P_redefine_weights(self.pruned_weights[-1],self.centers[-1])
        self.idx_layers = [idx_hidden,idx_output]
        
        # Setting time print    
        end_setting_time = dt.datetime.now() - start_setting_time
        eta = divmod(end_setting_time.days * 86400 + end_setting_time.seconds, 60)
        self.eta_print_setting = str(eta[0])+"m"+str(eta[1])+"s"
        if self.verbose:
            print("--- Setting Time: %s ---" % self.eta_print_setting)
    
 

    def train(self, training, testing):
        
        accu = [0.,0.]
        
        # Batch Setting
        len_batch_train = len(training[0])
        len_batch_test = len(testing[0])
        if(self.batchsize > 0 and self.batchsize <= 1):
            len_batch_train = int(np.ceil(len_batch_train * self.batchsize))
            len_batch_test = int(np.ceil(len_batch_test * self.batchsize))
        
        # Divide training and testing batches
        test_output = testing[0:len_batch_test][0:len_batch_test]
        inputs = training[0][0:len_batch_train]
        targets = np.zeros((len_batch_train, 10))
        for i in range(len_batch_train):
            targets[i, training[1][i]] = 1
        
        # Start prints 
        self.start_time = dt.datetime.now()
        print('-- Training Session Start (%s) --' % (self.start_time))
        typeTrainingPrint = "Stop Function: "    
        if self.stop_f == 0:
            typeTrainingPrint += str(self.stop_p)+" epochs"
        elif self.stop_f == 1:
            typeTrainingPrint += str(self.stop_p)+" epoch(s) w/o improvements"
        elif self.stop_f == 2:
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        print('\nNeurons: %d\nBatch Train: %d (%d%%)\nBatch Test: %d (%d%%)\nClusters: %d\nPercentile: %d\n%s\n' % (self.neurons,len_batch_train,self.batchsize*100,len_batch_test,self.batchsize*100,self.cluster,self.percentile,typeTrainingPrint))
        
        # Performs iterations
        while not self.is_stop_function_enabled(accu[1]):
            
            # Backpropagate with feed forward
            for input_vector, target_vector in zip(inputs, targets):
                weights = []
                for i,c in zip(self.idx_layers,self.centers):
                    w = P_idx_matrix_to_matrix(i,c)
                    weights.append(w)
                self.backpropagate(input_vector, target_vector, weights)
                
            # Accuracy
            accu = self.accu(test_output,weights)
            self.iteration += 1
            
            # Messages
            self.print_message_iter(self.iteration,accu,self.ETAepoch(self.start_time))
                      
        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))

    def feed_forward(self, input_vector, weights):
        outputs = []
        for w in weights:
            input_with_bias = np.append(input_vector, 1)   # Ajout constante
            output = np.inner(w.toarray(), input_with_bias)
            output = special.expit(output) # Sigmoid function
            outputs.append(output)
            # The output is the input of the next layer
            input_vector = output
        return outputs

    def backpropagate(self, input_vector, target, weights):
        c = 1./math.sqrt(self.iteration + 10)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector, weights)

        # Calculation of partial derivatives for the output layer and subtraction
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        gradient = np.outer(output_deltas, np.append(hidden_outputs, 1))
        cg = P_centroid_gradient_matrix(self.idx_layers[-1],gradient,self.cluster)
        self.centers[-1] = self.centers[-1] - c * np.array(cg).reshape(self.cluster,1)

        # Calculation of partial derivatives for the hidden layer and subtraction
        cleared_bias = np.delete(weights[-1].toarray(),300,1).T #np.delete(weights[-1], 300, 1).T
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * np.dot(cleared_bias, output_deltas)
        gradient = np.outer(hidden_deltas, np.append(input_vector, 1))
        cg = P_centroid_gradient_matrix(self.idx_layers[0],gradient,self.cluster)
        self.centers[0] = self.centers[0] - c * np.array(cg).reshape(self.cluster,1)
        
    
    
    def predict(self, input_vector, weights):
        return self.feed_forward(input_vector,weights)[-1]

    def predict_one(self, input_vector, weights):
        return np.argmax(self.feed_forward(input_vector,weights)[-1])

    def accu(self, testing, weights):
        res = np.zeros((10, 2))
        for k in range(len(testing[1])):
            if self.predict_one(testing[0][k], weights) == testing[1][k]:
                res[testing[1][k]] += 1
            else:
                res[testing[1][k]][1] += 1
        total = np.sum(res, axis=0)
        each = [res[k][0]/res[k][1] for k in range(len(res))]
        min_c = sorted(range(len(each)), key=lambda k: each[k])[0]
        return np.round([each[min_c]*100, total[0]/total[1]*100, min_c], 2)
    
    
    def is_stop_function_enabled(self,accuracy):
        if self.stop_f == 0:
            if self.iteration < self.stop_p:
                return False
            else:
                return True
        elif self.stop_f == 1:
            if accuracy > self.best or self.iteration == 0:
                self.same = 0
                self.best = accuracy
                return False
            else:
                self.same += 1
                if self.same < self.stop_p:
                    return False
                else:
                    return True
        elif self.stop_f == 2:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
    
    def print_message_iter(self,iteration,accu,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "
        message += 'Accuracy: '+str(accu[1]).zfill(4)+'%\tMin: '+ str(accu[0]).zfill(4)+ '% ('+str(int(accu[2]))+')'
        print(message)
        
    def getWeights(self):
        return self.layers
    
    def minsec2sec(self,time):
        if 'm' in time:
            splitted = time.split('m')
            return int(splitted[0]) * 60 + int(splitted[1][:-1])
        else:
            return int(time[:-1])
    
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret

In [22]:
pre_trained_weights = nn.getWeights()
nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=256,pre_weights=pre_trained_weights,percentile=60,verbose=True,stop_function=1,stop_parameter=2)
nn_pr_km.train(TRAINING,TESTING)

--- Setting Time: 0m1s ---
-- Training Session Start (2018-11-18 12:07:35.802561) --

Neurons: 300
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Clusters: 256
Percentile: 60
Stop Function: 2 epoch(s) w/o improvements

Epoch 001 (3m49s)  Accuracy: 72.56%	Min: 30.49% (5)
Epoch 002 (7m36s)  Accuracy: 76.78%	Min: 49.1% (5)
Epoch 003 (11m23s) Accuracy: 79.47%	Min: 54.37% (5)
Epoch 004 (15m9s)  Accuracy: 79.27%	Min: 53.81% (5)
Epoch 005 (18m55s) Accuracy: 80.15%	Min: 57.4% (5)
Epoch 006 (22m41s) Accuracy: 80.14%	Min: 57.29% (5)
Epoch 007 (26m27s) Accuracy: 80.74%	Min: 64.13% (5)
Epoch 008 (30m14s) Accuracy: 81.05%	Min: 59.53% (5)
Epoch 009 (33m59s) Accuracy: 81.46%	Min: 64.91% (5)
Epoch 010 (37m46s) Accuracy: 81.22%	Min: 60.43% (5)
Epoch 011 (41m32s) Accuracy: 81.19%	Min: 59.42% (5)

-- Training Session End (2018-11-18 12:49:07.812451) --


In [23]:
pre_trained_weights = nn.getWeights()
nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=256,pre_weights=pre_trained_weights,percentile=75,verbose=True,stop_function=1,stop_parameter=2)
nn_pr_km.train(TRAINING,TESTING)

--- Setting Time: 0m1s ---
-- Training Session Start (2018-11-18 12:49:09.512794) --

Neurons: 300
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Clusters: 256
Percentile: 75
Stop Function: 2 epoch(s) w/o improvements

Epoch 001 (3m26s)  Accuracy: 10.28%	Min: 00.0% (0)
Epoch 002 (6m52s)  Accuracy: 10.28%	Min: 00.0% (0)
Epoch 003 (10m19s) Accuracy: 57.58%	Min: 00.0% (9)
Epoch 004 (13m45s) Accuracy: 59.07%	Min: 00.1% (9)
Epoch 005 (17m12s) Accuracy: 58.92%	Min: 00.0% (6)
Epoch 006 (20m38s) Accuracy: 58.91%	Min: 00.0% (6)

-- Training Session End (2018-11-18 13:09:48.140424) --


In [24]:
pre_trained_weights = nn.getWeights()
nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=256,pre_weights=pre_trained_weights,percentile=80,verbose=True,stop_function=1,stop_parameter=2)
nn_pr_km.train(TRAINING,TESTING)

--- Setting Time: 0m7s ---
-- Training Session Start (2018-11-18 13:09:55.967590) --

Neurons: 300
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Clusters: 256
Percentile: 80
Stop Function: 2 epoch(s) w/o improvements

Epoch 001 (3m50s)  Accuracy: 18.67%	Min: 00.0% (0)
Epoch 002 (7m40s)  Accuracy: 18.25%	Min: 00.0% (3)
Epoch 003 (11m30s) Accuracy: 18.25%	Min: 00.0% (3)

-- Training Session End (2018-11-18 13:21:26.526439) --


In [25]:
pre_trained_weights = nn.getWeights()
nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=256,pre_weights=pre_trained_weights,percentile=90,verbose=True,stop_function=1,stop_parameter=2)
nn_pr_km.train(TRAINING,TESTING)

--- Setting Time: 0m2s ---
-- Training Session Start (2018-11-18 13:21:28.587445) --

Neurons: 300
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Clusters: 256
Percentile: 90
Stop Function: 2 epoch(s) w/o improvements

Epoch 001 (3m38s)  Accuracy: 54.54%	Min: 00.0% (9)
Epoch 002 (7m16s)  Accuracy: 62.56%	Min: 00.0% (8)
Epoch 003 (10m54s) Accuracy: 66.16%	Min: 00.0% (8)
Epoch 004 (14m33s) Accuracy: 63.86%	Min: 00.0% (8)
Epoch 005 (18m11s) Accuracy: 66.99%	Min: 00.0% (8)
Epoch 006 (21m50s) Accuracy: 66.7%	Min: 00.0% (8)
Epoch 007 (25m28s) Accuracy: 63.12%	Min: 00.0% (8)

-- Training Session End (2018-11-18 13:46:57.458809) --


In [26]:
pre_trained_weights = nn.getWeights()
nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=256,pre_weights=pre_trained_weights,percentile=94,verbose=True,stop_function=1,stop_parameter=2)
nn_pr_km.train(TRAINING,TESTING)

--- Setting Time: 0m1s ---
-- Training Session Start (2018-11-18 13:46:59.461193) --

Neurons: 300
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Clusters: 256
Percentile: 94
Stop Function: 2 epoch(s) w/o improvements

Epoch 001 (3m46s)  Accuracy: 51.31%	Min: 00.0% (3)
Epoch 002 (7m32s)  Accuracy: 51.55%	Min: 00.0% (8)
Epoch 003 (11m19s) Accuracy: 55.81%	Min: 00.0% (8)
Epoch 004 (15m6s)  Accuracy: 58.34%	Min: 00.0% (8)
Epoch 005 (18m53s) Accuracy: 58.25%	Min: 00.0% (8)
Epoch 006 (22m40s) Accuracy: 58.2%	Min: 00.0% (8)

-- Training Session End (2018-11-18 14:09:39.665675) --


In [27]:
pre_trained_weights = nn.getWeights()
nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=256,pre_weights=pre_trained_weights,percentile=98,verbose=True,stop_function=1,stop_parameter=2)
nn_pr_km.train(TRAINING,TESTING)

--- Setting Time: 0m2s ---
-- Training Session Start (2018-11-18 14:09:41.846824) --

Neurons: 300
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Clusters: 256
Percentile: 98
Stop Function: 2 epoch(s) w/o improvements

Epoch 001 (3m39s)  Accuracy: 64.02%	Min: 00.0% (8)
Epoch 002 (7m19s)  Accuracy: 65.91%	Min: 00.0% (8)
Epoch 003 (10m58s) Accuracy: 66.38%	Min: 00.0% (8)
Epoch 004 (14m37s) Accuracy: 66.8%	Min: 00.0% (8)
Epoch 005 (18m17s) Accuracy: 66.25%	Min: 00.0% (8)
Epoch 006 (21m57s) Accuracy: 67.14%	Min: 00.0% (8)
Epoch 007 (25m37s) Accuracy: 67.34%	Min: 00.0% (8)
Epoch 008 (29m16s) Accuracy: 67.98%	Min: 00.0% (8)
Epoch 009 (32m56s) Accuracy: 67.84%	Min: 00.0% (8)
Epoch 010 (36m36s) Accuracy: 67.7%	Min: 00.0% (8)

-- Training Session End (2018-11-18 14:46:17.916968) --
